# LineSetNode
The LineSetNode class in Geomapi represents the data and metadata of polygonal LineSet data. The data itself and methods build upon Open3D TriangleLineSet and TriLineSet concepts while the metadata builds upon the RDFlib framework:

[http://www.open3d.org/docs/latest/tutorial/Basic/LineSet.html#](http://www.open3d.org/docs/latest/tutorial/Basic/LineSet.html#) 

[https://trimsh.org/triLineSet.base.html](https://trimsh.org/triLineSet.base.html)

[https://rdflib.readthedocs.io/](https://rdflib.readthedocs.io/) 

The code below shows how to create a LineSetNode from various inputs.

First the geomapi and external packages are imported

In [1]:
#IMPORT PACKAGES
from rdflib import Graph
import os
import numpy as np

#IMPORT MODULES
from context import geomapi #context import for documentation only
from geomapi.nodes import LineSetNode

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## LineSetNode Creation

A LineSetNode is constructed using the same parameters as the base Node. Please refer to [Node Tutorial](../tutorial/tutorial_nodes.ipynb) For more info about Node Creation

In [ ]:
LineSetNode( subject = None,             # (URIRef, optional) : A subject to use as identifier for the Node.
             graph = None,               # (Graph, optional) : An RDF Graph to parse.
             graphPath = None,           # (Path, optional) : The path of an RDF Graph to parse.
             name = None,                # (str, optional) : A name of the Node.
             path = None,                # (Path, optional) : A filepath to a resource.
             timestamp = None,           # (str, optional) : Timestamp for the node.
             resource = None,            # (optional) : Resource associated with the node.
             cartesianTransform = None,  # (np.ndarray, optional) : The (4x4) transformation matrix.
             orientedBoundingBox = None, # (o3d.geometry.OrientedBoundingBox, optional) : The oriented bounding box of the node.
             convexHull = None,          # (o3d.geometry.TriangleLineSet, optional) : The convex hull of the node.
             loadResource = False,       # Load the resource at initialization?
             dxfPath = None,             # (str|Path) : path to DXF file
             handle = None,              # (str) : CAD handle
             layer = None,               # (str) : CAD layername e.g. IFC$1$BT8_Loofboom_Laag_WGI2, etc.
             dxfType = None              # (str, optional) : type of the object
            )

### Ontology link

The LineSetNode has 6 new standard properties that are serialized to the graph:

| python name | predicate |
|-----------  |-----------|
| `pointCount` | `geomapi:pointCount` |
| `lineCount` | `geomapi:lineCount` |
| `dxfPath` | `geomapi:dxfPath` |
| `handle` | `geomapi:handle` |
| `layer` | `geomapi:layer` |
| `dxfType` | `geomapi:dxfType` |


### Creation From dxf file

LineSetNodes can be created from autocad dfx files.
Since dxf files contain a large amount of elements, they cannot be loaded directly into a LineSetNode.
Use the [`geomapi.tools.dxf_to_lineset_nodes`](../geomapi/geomapi.tools.html#geomapi.tools.dxf_to_lineset_nodes) function to load all elements into a list of LineSetNode.

In [5]:
import geomapi.tools as tl

linesetnodes = tl.dxf_to_lineset_nodes(dxfPath = r"../../..\tests\testfiles\cad/railway.dxf")

Reading DXF file from ..\..\..\tests\testfiles\cad\railway.dxf...
162 entities were not LineSets. Skipping for now...
    loaded 121 lineSetNodes from dxf file


## LineSetNode Resource

When creating a Node with a resource, it can be done either directly with the resource, or with the path to the resource.

A resource can be a big piece of data, this is why it is not always wanted to load the whole resource at initialization. This is why the `loadResource` parameter is default to `False`

For more info on specific resources, see the corresponding Node type

### Loading The Resource

In [6]:
node = LineSetNode(path=r"../../..\tests\testfiles\cad\line.ply", loadResource=False)
print("resource before loading:",node.resource)
node.load_resource() # Use specialized node fo each type of resource.
print("resource after loading:",node.resource)

Resource not loaded, but path is defined, call `load_resource()` to access it.
Resource not loaded, but path is defined, call `load_resource()` to access it.
resource before loading: None
resource after loading: LineSet with 1 lines.


### Saving The Resource

A LineSet resource can be saved to disk using the `save_resource()` function.
Currently supports: .ply, .obj

In [9]:
node = LineSetNode(path=r"../../..\tests\testfiles\cad\line.ply", loadResource=True)
node.save_resource(directory=r"../../../tests/testfiles/resources", extension=".ply") # Save the resource to the resourcePath

True

## LineSetNode Transformation

Since every nod has a cartesian transform, it can be transformed using the `node.transform()` function.

The transformation also updates the `convexHull` and `orientedBoundingBox`.

Furthermore, if the LineSetNode has a resource, that resource is also transformed.

In [10]:
node = LineSetNode()
print(node.cartesianTransform)
transformation = np.array([[0,0,1,0],[0,1,0,0],[1,0,0,0],[0,0,0,1]])
node.transform(transformation=transformation)
print("applying transformation: (-1)")
print(node.cartesianTransform,"\n")

node = LineSetNode()
rotation = np.array([90,0,0]) #eulers in degrees
node.transform(rotation=rotation)
print("applying rotation: (90,0,0)")
print(node.cartesianTransform,"\n")

node = LineSetNode()
translation = np.array([1,2,3])
node.transform(translation=translation)
print("applying translation: (1,2,3)")
print(node.cartesianTransform)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
applying transformation: (-1)
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]] 

applying rotation: (90,0,0)
[[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]] 

applying translation: (1,2,3)
[[1. 0. 0. 1.]
 [0. 1. 0. 2.]
 [0. 0. 1. 3.]
 [0. 0. 0. 1.]]


## LineSetNode Visualisation

When a LineSetNode has a resource, the `show()` function displays the LineSet using either open3d or TriLineSet, depending on the workspace.

Use the `inline = True` parameter to display the LineSet using the TriLineSet viewer in your jupyter notebook file. Otherwise the function opens a new python window to display the open3d viewer

In [12]:
node = LineSetNode(path=r"../../..\tests\testfiles\cad\line.ply", loadResource=True)
node.show(inline=False) # The standard node has no resource to display

## Further reading

Please refer to the full [API documentation](../geomapi/geomapi.nodes.LineSetnode.rst) of the LineSetNode class for more details about the functionality